In [9]:
from typing import Any
import tmdbsimple as tmdb
import yfinance as yf
import pandas as pd
import datetime
import plotly.express as px
import plotly.graph_objects as go
from textblob import TextBlob

from my_secrets import TMDB_API_KEY

In [10]:
# The API key to fetch data from the TMDB
tmdb.API_KEY = TMDB_API_KEY

In [11]:
# Class with all the methods regarding the movie data and TMDB api
class MovieData:
    def __call__(self, company_id, num_movies=10) -> Any:
        company = tmdb.Companies(company_id)
        movies = company.movies()['results'][:num_movies]
        movie_details = []
        for movie in movies:
            movie_info = tmdb.Movies(movie['id']).info()
            reviews = tmdb.Movies(movie['id']).reviews()
            review_texts = [review['content'] for review in reviews['results']]
            movie_details.append({
                'title': movie_info['title'],
                'release_date': movie_info['release_date'],
                'description': movie_info['overview'],
                'reviews': ' '.join(review_texts)  # Combine all reviews into a single string
            })
        return pd.DataFrame(movie_details)

In [12]:
# Firstly, fetching the movie data for 10 movies of Walt Disney Pictures
movie_data = MovieData()(company_id=2)
movie_data

,title,release_date,description,reviews
0,Inside Out 2,2024-06-11,Teenager Riley's mind headquarters is undergoi...,"""Inside Out 2"" is a heartwarming and imaginati..."
1,Inside Out,2015-06-09,From an adventurous balloon ride above the clo...,This is the most incredible movie I've ever se...
2,Wish,2023-11-13,"Asha, a sharp-witted idealist, makes a wish so...",With her grandpa hitting his hundredth birthda...
3,Elemental,2023-06-14,"In a city where fire, water, land and air resi...",Finding your way in the world can be very diff...
4,Turning Red,2022-03-10,Thirteen-year-old Mei is experiencing the awkw...,The digital animation geniuses at Pixar are ba...
5,Coco,2017-10-27,Despite his family’s baffling generations-old ...,"Spectacularly colourful with a sweet, (albeit ..."
6,Encanto,2021-10-13,"The tale of an extraordinary family, the Madri...",In a magical area of Columbia surrounded by mo...
7,WALL·E,2008-06-22,"In the distant future, Earth has become a deso...",A movie about robots and emotions? Pixar has m...
8,Pirates of the Caribbean: The Curse of the Bla...,2003-07-09,"Jack Sparrow, a freewheeling 18th-century pira...","Ah, but you have heard of me.\r\n\r\nThe crew ..."
9,"The Chronicles of Narnia: The Lion, the Witch ...",2005-12-07,"Siblings Lucy, Edmund, Susan and Peter step th...","Long bloody title, but a great film no doubt.\..."


In [13]:
# Class with all the methods with relation to the stock market data
class StockData:
    def __call__(self,movie_data, ticker, days_before=15, days_after=15) -> Any:
        stock_data_list = []
        for release_date in movie_data['release_date']:
            release_date_dt = datetime.datetime.strptime(release_date, '%Y-%m-%d')
            start_date = (release_date_dt - datetime.timedelta(days=days_before)).strftime('%Y-%m-%d')
            end_date = (release_date_dt + datetime.timedelta(days=days_after)).strftime('%Y-%m-%d')
            stock_data = self.get_stock_data(ticker, start_date, end_date)
            stock_data['release_date'] = release_date_dt
            stock_data_list.append(stock_data)
        return stock_data_list

    def get_stock_data(self, ticker, start_date, end_date):
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        return stock_data

In [14]:
# Getting the stock prices for the time around the movie release (15 days before and after the release)
stock_data_list = StockData()(movie_data, ticker='DIS') # DIS stands for Disney in the stock market

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [15]:
stock_data_list[0].head()

,Open,High,Low,Close,Adj Close,Volume,release_date
Date,,,,,,,
2024-05-28,101.360001,102.860001,100.949997,102.440002,102.440002,7813100,2024-06-11
2024-05-29,101.760002,102.080002,100.769997,100.879997,100.879997,7376300,2024-06-11
2024-05-30,101.139999,102.070000,100.970001,101.699997,101.699997,7303700,2024-06-11
2024-05-31,101.519997,104.080002,101.410004,103.910004,103.910004,14735800,2024-06-11
2024-06-03,104.169998,104.220001,102.480003,102.769997,102.769997,6908600,2024-06-11


In [20]:
# Class method for performing sentiment analysis
class SentimentAnalysis:
    def reviews_sentiment(self, movie_data_sentiment:pd.DataFrame) -> Any:
        return movie_data_sentiment['reviews'].apply(self.analyze_sentiment)

    def description_sentiment(self, movie_data_sentiment:pd.DataFrame) -> Any:
        return movie_data_sentiment['description'].apply(self.analyze_sentiment)

    def analyze_sentiment(self, text):
        blob = TextBlob(text)
        return blob.sentiment.polarity

In [21]:
    # Analyzing the sentiment on movie reviews
movie_data['sentiment_reviews'] = SentimentAnalysis().reviews_sentiment(movie_data)

In [22]:
movie_data['sentiment_reviews']

0    0.150528
1    0.237601
2    0.189218
3    0.185440
4    0.149624
5    0.288305
6    0.325953
7    0.249368
8    0.174468
9    0.314684
Name: sentiment_reviews, dtype: float64

In [23]:
    # Analyzing the sentiment on movie description
movie_data['sentiment_description'] = SentimentAnalysis().description_sentiment(movie_data)

In [24]:
movie_data['sentiment_description']

0    0.196307
1    0.383333
2    0.152083
3    0.034091
4    0.125000
5    0.123611
6    0.216942
7   -0.005000
8    0.500000
9    0.350000
Name: sentiment_description, dtype: float64

In [28]:
# Class for visualization of the findings
class Visualization:
    def stock_plots(self,movie_data, stock_data_list) -> None:
        for i, release_date in enumerate(movie_data['release_date']):
            stock_data = stock_data_list[i]
            release_date_dt = datetime.datetime.strptime(release_date, '%Y-%m-%d')
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data['Close'], mode='lines+markers', name='Closing Price'))
            
            # Add vertical line for the release date
            fig.add_vline(x=release_date_dt, line=dict(color='red', dash='dash'))
            
            # Adding annotation for the release date
            fig.add_annotation(x=release_date_dt, y=stock_data['Close'].max(), text='Release Date', showarrow=True, arrowhead=2)

            fig.update_layout(
                title=f"Stock Prices Around Release Date of '{movie_data['title'][i]}'",
                xaxis_title='Date',
                yaxis_title='Stock Closing Price',
                template='plotly_dark'
            )
            fig.show()

    def sentiment_plot(self, movie_data, sentiment_on) -> None:
        fig = px.bar(movie_data[['title',f'sentiment_{sentiment_on}']], x='title', y=f'sentiment_{sentiment_on}', title=f'Sentiment for Each Movie Title on {sentiment_on}', labels={'sentiment': 'Sentiment Score', 'title': 'Movie Title'})
        fig.update_layout(xaxis_title='Movie Title', yaxis_title='Sentiment Score', template='plotly_dark')
        fig.show()

In [29]:
# Visualization
    # Visualizing the sentiments
visualizer = Visualization()
# Sentiments for reviews
visualizer.sentiment_plot(movie_data, "reviews")

In [30]:
# sentiments for description
visualizer.sentiment_plot(movie_data, "description")

In [31]:
# Visualization the graphs of stock prices and release dates for each movie
visualizer.stock_plots(movie_data, stock_data_list)